In [110]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import sign

def xi_generate(x_range,e):
    bin=len(x_range)
    C=np.random.random((bin,bin))
    for i in range(bin):
        for j in range(bin):
            C[i,j]=abs(x_range[i]-x_range[j])/e
    return C

def bisection(fun, a, b, stepmax, tol):
    if sign(fun(a)) == 0: return a
    if sign(fun(b)) == 0: return b
    if sign(fun(a)) * sign(fun(b)) > 0: return print("bad section")
    for step in range(1, stepmax+1):
        c = (a + b) / 2
        if sign(fun(c)) == 0: return c
        if abs((b - a)/2) < tol: return (b + a) / 2
        if sign(fun(c)) * sign(fun(a)) < 0:
            b = c
        else:
            a = c
    return (a + b) / 2  

def tmp_generator(gamma_dict,num,q_dict,q_num,L):
    bin=gamma_dict[0].shape[0]
    if q_num<=0:
        q=np.matrix(np.ones((bin,bin)))
    else:
        q=q_dict[q_num]
    tmp_gamma=np.zeros((bin,bin))
    tmp_q=np.zeros((bin,bin))
    for i in range(bin):
        for j in range(bin):
            tmp_gamma[i,j]=q.item(i,j)*gamma_dict[num-1].item(i,j)*gamma_dict[num-L-1].item(i,j)/gamma_dict[num-L].item(i,j)
            tmp_q[i,j]=q.item(i,j)*gamma_dict[num-L-1].item(i,j)/gamma_dict[num-L].item(i,j)
    return np.matrix(tmp_gamma),np.matrix(tmp_q)     

def Gaussian_pdf(x,name,para):
    mu=para[name+'_mean']
    sigma=para[name+'_sd']
    return math.exp(((x-mu)/sigma)**2*(-1/2))/(sigma*math.sqrt(2*math.pi))  
def empirical_distribution(sub,dist):
    bin=dist['bin']
    distrition=np.zeros(bin)
    for i in range(bin):
        subset=sub[sub['X']==i] #bin_value=x_range[i] #sub[(sub['X']>=bin_value)&(sub['X']<bin_value+width)]
        if subset.shape[0]>0:
            distrition[i]=sum(subset['W'])
    if sum(distrition)>0:
        return distrition/sum(distrition)
    else:
        return distrition

def data_generation(num,para):
    df=pd.DataFrame(columns=['X', 'S'])
    for ignore in range(num):
        if np.random.random()<para['f']:
            X=np.floor((np.random.normal(para['X_f_mean'],para['X_f_sd'],size=[1])-horizen[0])/width)
            sample=[X,'f']
        else:
            X=np.floor((np.random.normal(para['X_m_mean'],para['X_m_sd'],size=[1])-horizen[0])/width)
            sample=[X,'m']
        df=pd.concat([df, pd.DataFrame(sample, index=['X','S']).T], ignore_index=True)
    df['W']=1
    df['X']=df['X'].astype('int64')  
    return df

def samples_groupby(data):
    # for better complexity
    df=data.groupby(by=['X','S'],as_index=False).sum()
    return df[df['W']!=0]

def projection(df,coupling):
    df_t=pd.DataFrame(columns=['X','S','W'])
    for i in range(df.shape[0]):
        orig=df.iloc[i]
        rows=np.nonzero(coupling[:,orig[0]])[0]
        sub=pd.DataFrame(columns=['X','W'],index=rows)
        sub['X']=rows
        sub['W']=coupling[rows,orig[0]]/(sum(coupling[rows,orig[0]])+0.0001)*orig[2]
        sub['S']=orig[1]
        df_t=pd.concat([df_t, samples_groupby(sub)], ignore_index=True)
    return df_t

n: the number of points.

In [148]:
gamma_dict[0].shape[0]

50

In [144]:
para = dict()
para['X_0_mean']=-8
para['X_0_sd']=7
para['X_1_mean']=2
para['X_1_sd']=3
para['X_t_mean']=-10
para['X_t_sd']=5
para['s_0']=0.6
para['s_1']=0.4

horizen=[-30,20]
width=1
bin=int((horizen[1]-horizen[0])/width)
x_range=np.arange(horizen[0],horizen[1],width)
dist = dict(zip(['horizen','width','bin','x_range'],[horizen,width,bin,x_range]))
dist['x_0']=[Gaussian_pdf(x,'X_0',para)*width for x in x_range]
dist['x_1']=[Gaussian_pdf(x,'X_1',para)*width for x in x_range]
dist['x']=[dist['x_0'][i]*para['s_0']+dist['x_1'][i]*para['s_1'] for i in range(bin)]
dist['t_x']=[Gaussian_pdf(x,'X_t',para)*width for x in x_range] 
dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]

e=0.01
xi=xi_generate([*range(bin)],e)

# simplist
gamma=xi
for repeat in range(100):
    new_gamma=np.diag(dist['x']/sum(gamma.T))@gamma
    new2_gamma=new_gamma@np.diag(dist['t_x']/sum(new_gamma))
    gamma=new2_gamma

# our method
bbm1=np.matrix(np.ones(bin)).T
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T
V=np.matrix(dist['v']).T
I=np.where(~(V==0))[0].tolist() # \overline{\supp(X)}

gamma_dict=dict()
gamma_dict[0]=np.matrix(xi+0.01)
gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
# step 3
J=np.where(~((gamma_dict[2].T @ V).A1 ==0))[0].tolist()
nu=np.zeros(bin)
gamma_dict[3]=np.copy(gamma_dict[2])
for j in J:
    fun = lambda z: sum(gamma_dict[2].item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
    nu = bisection(fun, -5, 5, stepmax = 25, tol = 1.0e-3)
    for i in I:
        gamma_dict[3][i,j]=np.exp(nu*V.item(i))*gamma_dict[2].item(i,j)
gamma_dict[3]=np.matrix(gamma_dict[3])

#=========================
L=3
q_dict=dict()
for loop in range(1,50):
    tmp,q_dict[(loop-1)*L+1]=tmp_generator(gamma_dict,loop*L+1,q_dict,(loop-2)*L+1,L) #np.matrix(gamma_dict[3].A1*gamma_dict[0].A1/gamma_dict[1].A1)
    gamma_dict[loop*L+1]=np.matrix(np.diag((px/(tmp @ bbm1)).A1))@tmp

    tmp,q_dict[(loop-1)*L+2]=tmp_generator(gamma_dict,loop*L+2,q_dict,(loop-2)*L+2,L)  #np.matrix(gamma_dict[4].A1*gamma_dict[1].A1/gamma_dict[2].A1)
    gamma_dict[loop*L+2]=tmp@np.matrix(np.diag((ptx/(tmp.T @ bbm1)).A1))

    # step 3
    tmp,q_dict[(loop-1)*L+3]=tmp_generator(gamma_dict,loop*L+3,q_dict,(loop-2)*L+3,L)  #np.matrix(gamma_dict[5].A1*gamma_dict[2].A1/gamma_dict[3].A1)
    J=np.where(~((np.matrix(tmp).T @ V).A1 ==0))[0].tolist()
    gamma_dict[loop*L+3]=np.copy(tmp)
    for j in J:
        fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
        nu = bisection(fun, -5, 5, stepmax = 25, tol = 1.0e-3)
        for i in I:
            gamma_dict[loop*L+3][i,j]=np.exp(nu*V.item(i))*tmp.item(i,j)
    gamma_dict[loop*L+3]=np.matrix(gamma_dict[loop*L+3])


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 50 is different from 4)

In [140]:
np.where(~(V==0))[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype=int64)

In [143]:
dist['v']==0

True

In [137]:
np.where(~(dist['v']==0))[0]

array([0], dtype=int64)

In [128]:
gamma_dict[2].T 

matrix([[1.39403602e-14, 2.25027066e-10, 7.11625420e-10, ...,
         1.68916822e-09, 9.82784360e-10, 5.60863328e-10],
        [3.16269734e-10, 5.10424536e-14, 8.07204145e-10, ...,
         3.75036130e-09, 2.18300680e-09, 1.24635693e-09],
        [1.38076034e-09, 1.11436473e-09, 1.76194499e-13, ...,
         8.00903517e-09, 4.66409590e-09, 2.66410576e-09],
        ...,
        [1.13136150e-11, 1.78722293e-11, 2.76466073e-11, ...,
         6.20587097e-18, 3.53580367e-14, 3.95312375e-14],
        [3.67083655e-12, 5.80148299e-12, 8.97857531e-12, ...,
         1.97181633e-14, 1.12322046e-18, 6.27989549e-15],
        [1.14622643e-12, 1.81231185e-12, 2.80606666e-12, ...,
         1.20621657e-14, 3.43604609e-15, 1.92070000e-19]])

In [124]:
J=np.where(~((gamma_dict[2].T @ V.T).A1 ==0))[0].tolist()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 50)

In [9]:
dist=dict()
dist['s']=np.array([0.6,0.4])
dist['x_0']= (1/n)*np.ones(n)
dist['x_1']= np.concatenate(((1/(n-2))*np.ones(n-2),np.zeros(2)),axis=None)
dist['x']=dist['s'][0]*dist['x_0']+dist['s'][1]*dist['x_1']
dist['t_x']= (1/n)*np.ones(n)

In [10]:
v=(dist['x_0']-dist['x_1'])/dist['x']

In [11]:
e=0.01

sum(gamma) == gamma.T \mathbb{1}

sum(gamma.T) == gamma \mathbb{1}

In [118]:
xi=xi_generate([*range(bin)],e)

In [14]:
theta=abs(gamma.T @ v)*(1-0.3)

In [93]:
I=np.where(~(v==0))[0].tolist() # \overline{\supp(X)}
bbm1=np.matrix(np.ones(n)).T
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T
V=np.matrix(v).T
Ve=np.multiply(V,np.exp(V))
#theta=np.matrix(abs(gamma.T @ v)*(1-0.3)).T
gamma_dict=dict()
gamma_dict[0]=np.matrix(xi+0.01)
gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
# step 3
J=np.where(~((np.matrix(gamma_dict[2]).T @ np.matrix(v).T).A1 ==0))[0].tolist()
nu=np.zeros(n)
gamma_dict[3]=np.copy(gamma_dict[2])
for j in J:
    fun = lambda z: sum(gamma_dict[2].item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
    nu = bisection(fun, -5, 5, stepmax = 25, tol = 1.0e-3)
    for i in I:
        gamma_dict[3][i,j]=np.exp(nu*V.item(i))*gamma_dict[2].item(i,j)
gamma_dict[3]=np.matrix(gamma_dict[3])

#=========================
L=3
q_dict=dict()
for loop in range(1,50):
    tmp,q_dict[(loop-1)*L+1]=tmp_generator(gamma_dict,loop*L+1,q_dict,(loop-2)*L+1,L) #np.matrix(gamma_dict[3].A1*gamma_dict[0].A1/gamma_dict[1].A1)
    gamma_dict[loop*L+1]=np.matrix(np.diag((px/(tmp @ bbm1)).A1))@tmp

    tmp,q_dict[(loop-1)*L+2]=tmp_generator(gamma_dict,loop*L+2,q_dict,(loop-2)*L+2,L)  #np.matrix(gamma_dict[4].A1*gamma_dict[1].A1/gamma_dict[2].A1)
    gamma_dict[loop*L+2]=tmp@np.matrix(np.diag((ptx/(tmp.T @ bbm1)).A1))

    # step 3
    tmp,q_dict[(loop-1)*L+3]=tmp_generator(gamma_dict,loop*L+3,q_dict,(loop-2)*L+3,L)  #np.matrix(gamma_dict[5].A1*gamma_dict[2].A1/gamma_dict[3].A1)
    J=np.where(~((np.matrix(tmp).T @ np.matrix(v).T).A1 ==0))[0].tolist()
    gamma_dict[loop*L+3]=np.copy(tmp)
    for j in J:
        fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
        nu = bisection(fun, -5, 5, stepmax = 25, tol = 1.0e-3)
        for i in I:
            gamma_dict[loop*L+3][i,j]=np.exp(nu*V.item(i))*tmp.item(i,j)
    gamma_dict[loop*L+3]=np.matrix(gamma_dict[loop*L+3])


In [94]:
gamma_dict

{0: matrix([[1.0000e-02, 1.0001e+02, 2.0001e+02, 3.0001e+02],
         [1.0001e+02, 1.0000e-02, 1.0001e+02, 2.0001e+02],
         [2.0001e+02, 1.0001e+02, 1.0000e-02, 1.0001e+02],
         [3.0001e+02, 2.0001e+02, 1.0001e+02, 1.0000e-02]]),
 1: matrix([[5.83294447e-06, 5.83352776e-02, 1.16664722e-01, 1.74994167e-01],
         [8.75000000e-02, 8.74912509e-06, 8.75000000e-02, 1.74991251e-01],
         [7.49962504e-02, 3.75000000e-02, 3.74962504e-06, 3.75000000e-02],
         [7.49975002e-02, 4.99991667e-02, 2.50008333e-02, 2.49983334e-06]]),
 2: matrix([[6.13995232e-06, 9.99965721e-02, 1.27269141e-01, 1.12902983e-01],
         [9.21054247e-02, 1.49974861e-05, 9.54534464e-02, 1.12901101e-01],
         [7.89435599e-02, 6.42813681e-02, 4.09045294e-06, 2.41943028e-02],
         [7.89448755e-02, 8.57070624e-02, 2.72733223e-02, 1.61284599e-06]]),
 3: matrix([[9.30258207e-06, 1.45547445e-01, 8.74386892e-02, 7.45189804e-02],
         [1.39548034e-01, 2.18292062e-05, 6.55801098e-02, 7.45177386e-0

In [99]:
#test:

In [100]:
print(gamma_dict[150]@bbm1)
print(px)

[[0.35]
 [0.35]
 [0.15]
 [0.15]]
[[0.35]
 [0.35]
 [0.15]
 [0.15]]


In [101]:
print(gamma_dict[150].T@bbm1)
print(ptx)

[[0.25]
 [0.25]
 [0.25]
 [0.25]]
[[0.25]
 [0.25]
 [0.25]
 [0.25]]


In [102]:
print(gamma_dict[150].T@V)

[[-1.05785689e-04]
 [ 1.52014323e-04]
 [-1.23015653e-04]
 [ 7.67870191e-05]]


In [ ]:

def algorithms(reg,m,n,g,f,C):
    K=np.exp(-C/C.max()/reg)
    interations=100
    trial=10 #check the optimiality every trial. 
    u=np.ones((n,1))
    u_record=np.ones((n,1))
    for tri in range(trial):
        if sum(abs(u_record-u))<10**(-10):
            for i in range(1,interations):
                v=g/np.dot(K.T,u)
                u=f/np.dot(K,v)
            u_record=u
        else:
            break
    return np.dot(np.diag(u.reshape((1,-1))[0]),np.dot(K,np.diag(v.reshape((1,-1))[0])))

def assess(m,n,g,f,C,output):
    print('sum of violation of f:',sum(abs(np.sum(output,1)-f.reshape(n))))
    print('sum of violation of g:',sum(abs(np.sum(output,0)-g.reshape(m))))
    print('total cost:',sum(sum(output*C)))
    print('entropy:',sum(sum(-output*np.log(output+0.1**3))))
    print('============================================')

def plots(x_range,g,f,output):
    fig = plt.figure(figsize=(3,3))
    gs = fig.add_gridspec(2, 2, width_ratios=(4,1), height_ratios=(1,4),left=0.1,right=0.9,bottom=0.1, top=0.9,wspace=0,hspace=0)
    # Create the Axes.
    ax = fig.add_subplot(gs[1, 0])
    ax.pcolormesh(x_range, x_range, output, cmap='Blues')
    ax.set_xlabel(r'$\mathbb{X}$',fontsize=14)
    ax.set_ylabel(r'$\tilde{\mathbb{X}}$',rotation='horizontal',fontsize=14)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax) 
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
    #ax_histx.set_title(r'$Pr[x]$',rotation='horizontal')
    #ax_histy.set_title(r'$Pr[\tilde{x}]$')
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histx.tick_params(axis="y", labelleft=False)
    ax_histy.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    ax_histx.plot(x_range,g,color='tab:blue')
    ax_histy.plot(f,x_range,color='tab:green') 
    return fig

def OT(reg,x_range,g,f):
    m=len(g)
    n=len(f)
    C=c_generate(x_range)
    output=algorithms(reg,m,n,g.reshape(m,-1),f.reshape(n,-1),C)
    assess(m,n,g,f,C,output)
    plots(x_range,g,f,output)
    return outputa